In [81]:
#!jupyter nbextension enable --py gmaps

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [82]:
# Store csv created in part one into a DataFrame
weather = "output_data\my_weather_data.csv"

# Read the weather data and the study results
weather_data = pd.read_csv(weather)

weather_data.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Albany,42.6001,-73.9662,10.00,62,100,6.19,US,1616985963
1,1,Port Elizabeth,-33.9180,25.5701,18.00,77,0,6.17,ZA,1616985963
2,2,Martapura,-3.4167,114.8500,30.00,74,20,1.03,ID,1616985963
3,3,Lebu,-37.6167,-73.6500,13.22,91,25,4.00,CL,1616985964
4,4,Rikitea,-23.1203,-134.9692,26.51,71,1,6.54,PF,1616985964


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [83]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [84]:
# Heatmap of humidity
location = weather_data[["Latitude", "Longitude"]].astype(float)

#weatherdf = weatherdf.dropna()

humidity_rate = weather_data["Humidity"].astype(float)

In [85]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#used metric measurements instead of imperial. Converted 80 and 70 degrees F to celsius
new_clean_data = weather_data.loc[(weather_data["Max Temp"] <27) & (weather_data["Max Temp"] >21) &
                                  (weather_data["Wind Speed"] <10) & (weather_data["Cloudiness"] ==0)]

new_clean_data

,Unnamed: 0,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,Saint-Philippe,-21.3585,55.7679,25.00,69,0,3.09,RE,1616985964
41,41,Saint-Pierre,-21.3393,55.4781,25.00,69,0,3.09,RE,1616985973
156,156,Araouane,18.9048,-3.5265,22.15,8,0,4.86,ML,1616986004
180,180,Mildura,-34.2000,142.1500,23.89,38,0,3.09,AU,1616986011
260,260,Murray Bridge,-35.1167,139.2667,22.22,54,0,2.48,AU,1616986031
290,290,Birao,10.2849,22.7882,23.88,11,0,5.10,CF,1616986039
341,341,Salalah,17.0151,54.0924,23.00,43,0,1.03,OM,1616986054
390,390,Road Town,18.4167,-64.6167,25.56,83,0,8.05,VG,1616985846
477,477,Vila Velha,-20.3297,-40.2925,24.00,88,0,2.06,BR,1616986088
484,484,Saint-Paul,-21.0096,55.2707,25.00,78,0,2.06,RE,1616986090


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [87]:
# Create DataFrame called hotel_df to store hotel names along with city, country and coordinates
hotel_df = new_clean_data[["City","Country","Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""

<ipython-input-87-c3aaea2db13a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


In [88]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
6,Saint-Philippe,RE,-21.3585,55.7679,
41,Saint-Pierre,RE,-21.3393,55.4781,
156,Araouane,ML,18.9048,-3.5265,
180,Mildura,AU,-34.2000,142.1500,
260,Murray Bridge,AU,-35.1167,139.2667,
290,Birao,CF,10.2849,22.7882,
341,Salalah,OM,17.0151,54.0924,
390,Road Town,VG,18.4167,-64.6167,
477,Vila Velha,BR,-20.3297,-40.2925,
484,Saint-Paul,RE,-21.0096,55.2707,


In [89]:
# Set parameters to search for a hotel

for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    country = row['Country']
    
    # Use the search term: "Hotel" and our lat/lng
    print(f"Retrieving Results for {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results']['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Saint-Philippe.


TypeError: list indices must be integers or slices, not str

In [90]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [91]:
# Add marker layer ontop of heat map

fig = gmaps.figure

heat_layer = gmaps.heatmap_layer(locations, weights=humidity_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1, opacity=1)

marker_layer = gmaps.marker_layer(heat_layer)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#plt.savefig("vacationheatmapwithhotels.png")

fig
# Display figure


ValueError: weights must be of the same length as locations or None

In [ ]:
#The first conclusion that I can draw from this assignment is that we can see humidity is higher when closer to a 
#body of water.  The second thing is that with respect to latitude, max temperature increases the closer to the equator
# (0 latitude) is, and lowers as it gets closer to the poles.  The last thing is that there is not a clear correlation
#between latitude and cloudiness.  Latitude doesn't really impact the formation of clouds, like it does with temperature.